In [1]:
import sqlite3

In [4]:
conn = sqlite3.connect('db.slqite3')

In [5]:
cursor = conn.cursor()

In [6]:
sql = """SELECT
	typeof(100),
	typeof(10.0),
	typeof('100'),
	typeof(x'1000'),
	typeof(NULL);
"""

cursor.execute(sql)

In [7]:
cursor.fetchall()

[('integer', 'real', 'text', 'blob', 'null')]

In [17]:
sql = """
create table if not exists products (
    id integer primary key autoincrement,
    name text not null,
    unit text not null,
    type text not null
);

create table if not exists price (
    id integer primary key autoincrement,
    period datetime not null,
    price number not null,
    productId integer,

    foreign key (productId)
        references products (id)

);

create table if not exists stock (
    id integer primary key autoincrement,
    period datetime not null,
    qty number not null,
    productId integer,

    foreign key (productId)
        references products (id)

);
"""

In [18]:
cursor.executescript(sql)

In [21]:
#для створення товарів
sql = "insert into products (name, unit, type) values ('Клей', 'шт', 'буд.матеріали')"

cursor.execute(sql)
conn.commit()


cursor.execute('SELECT * FROM products')
cursor.fetchall()

[(1, 'Цемент', 'мішки', 'буд.матеріали'),
 (2, 'Плитка', 'шт', 'буд.матеріали'),
 (3, 'Клей', 'шт', 'буд.матеріали')]

In [24]:
#для створення цін
from datetime import datetime, timedelta
sql = "insert into price (period, price, productId) values (?, ?, ?)"

cursor.execute(
    sql, 
    (datetime.now() -timedelta(days=60), 50, 3)
)
conn.commit()


cursor.execute('SELECT * FROM price')
cursor.fetchall()

[(1, '2023-03-11 14:51:08.376875', 100, 3),
 (2, '2023-02-09 14:51:48.285808', 95, 3),
 (3, '2023-01-10 14:51:55.338228', 50, 3)]

In [33]:
#для створення цін
from datetime import datetime, timedelta
sql = "insert into stock (period, qty, productId) values (?, ?, ?)"

cursor.execute(
    sql, 
    (datetime.now() , -2, 3)
)
conn.commit()


cursor.execute('SELECT * FROM stock')
cursor.fetchall()

[(1, '2023-03-11 14:53:47.204549', 100, 3),
 (2, '2023-03-11 14:53:57.518118', -3, 3),
 (3, '2023-03-11 14:54:03.638823', -5, 3),
 (4, '2023-03-11 14:54:09.681805', -4, 3),
 (5, '2023-03-11 14:54:16.987024', -1, 3),
 (6, '2023-03-11 14:54:20.605463', -2, 3),
 (7, '2023-03-11 14:54:28.630434', -5, 3),
 (8, '2023-03-11 14:55:14.221532', -2, 3)]

In [34]:
cursor.execute('SELECT sum(qty) FROM stock')
cursor.fetchall()

[(78,)]

In [35]:
cursor.execute('create view inStock as SELECT productId, sum(qty) FROM stock GROUP BY productId')

cursor.execute('SELECT * FROM inStock')
cursor.fetchall()

[(3, 78)]